## W.  Wrangling The Pronto Cycle Share datasets

### W. 1 Reading in the datasets

The Pronto Seattle cycle share data is available in 3 separate CSV files.
1. Station.csv contains data about the 58 Pronto cycle docks around Seattle - https://www.kaggle.com/pronto/cycle-share-dataset/data
2. Trip.csv contains data about every trip taken using the Pronto Cycle share system in Seattle from 2014 to 2016 - https://www.kaggle.com/pronto/cycle-share-dataset/data
3. Weather.csv contains data about the weather in Seattle that corresponds to the dates of the recorded trips in Seattle in the Trip.csv file - https://www.kaggle.com/pronto/cycle-share-dataset/data

Each of the 3 CSV files have been read into separate pandas dataframes. 

The CSV files are freely available to anyone who wants to use them on Kaggle Datasets - https://www.kaggle.com/datasets

In [166]:
import pandas as pd

#Importing and reading csv files directly into pandas dataframe

station_data = pd.read_csv('/Users/sam/Data Science/Datasets/station.csv')

trip_data = pd.read_csv('/Users/sam/Data Science/Datasets/trip.csv', error_bad_lines=False)

weather_data = pd.read_csv('/Users/sam/Data Science/Datasets/weather.csv')

b'Skipping line 50794: expected 12 fields, saw 20\n'


**Note: The trip.csv file contained a corrupted line which had to be skipped for the data to be successfully read into pandas DataFrame**

### W.2 Cleaning the Station Data Set 


In [168]:
#Information about the various colums in the station_data DataFrame
station_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 9 columns):
station_id           58 non-null object
name                 58 non-null object
lat                  58 non-null float64
long                 58 non-null float64
install_date         58 non-null object
install_dockcount    58 non-null int64
modification_date    17 non-null object
current_dockcount    58 non-null int64
decommission_date    4 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 4.2+ KB


 - **The station dataset contains a column for modification_date of a station which contains the date on which the station dock was modified to hold more or less cycles and null values if the station was never modified after it was initially set up. I will drop this column entirely as it contains only 17 non_null values and does not provide us with information that would be useful for our analysis.**


- **The dataset also contains an install_dockcount column which contains the cycle holding capacity of the dock at a station when it was initially set up and the current_dockcount column holds the current cycle holding capacity of a dock at a station. The current_dockcount and the install_dockcount values for a station are the same if it was never modified. Since we are only interested in current values for all stations, I will drop the install_dockcount column entirely.**


- **4 out of the 58 stations in Seattle have been decommsioned and are no longer active. I will drop these decommisioned stations from the dataset , to work with only the active stations which also makes the decommissoned_date column unecessary for our analysis and I will be able to drop it as well.**

In [169]:
#Dropping the modification_date and install_dockcount columns from the dataset
station_data = station_data.drop(['modification_date','install_dockcount'],axis=1)

In [170]:
#Extracting only those stations that were not decommisoned during the time this data was collected
active_stations = station_data['decommission_date'].isnull()
station_data = station_data.loc[active_stations]

In [171]:
#Dropping the decommision_date column from the station_data DataFrame 
station_data = station_data.drop(['decommission_date'],axis=1)

In [172]:
#Information about the columns in the cleaned station_data Dataframe
station_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 57
Data columns (total 6 columns):
station_id           54 non-null object
name                 54 non-null object
lat                  54 non-null float64
long                 54 non-null float64
install_date         54 non-null object
current_dockcount    54 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.0+ KB


#### Exploring the Cleaned station_data Dataframe

In [173]:
#The first five rows of the cleaned Dataframe
station_data.head(5)

,station_id,name,lat,long,install_date,current_dockcount
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,10/13/2014,18
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,10/13/2014,16
2,BT-04,6th Ave & Blanchard St,47.616094,-122.341102,10/13/2014,16
3,BT-05,2nd Ave & Blanchard St,47.613110,-122.344208,10/13/2014,14
4,CBD-03,7th Ave & Union St,47.610731,-122.332447,10/13/2014,20


### W.2 Cleaning the Trip Dataset

The trip dataset contains some null values only in 2 columns, namely 'gender' and 'birthyear'. Furthermore these null values only occur when the trip belongs to Short Term Passholder , which ranges from a 24 hour pass or a 3 day pass as offered by Pronto as compared to a trip made by someone who is a Member, which could be a monthly subscription or a yearly one to the Pronto Cycle Share.

For the purpose of cleaning the null values and for a more precise analysis, I will split the trip data set on the basis of the client being a Member or a Temporary Pass Holder into 2 separate datasets. 

In [174]:
#Information about the various columns in the trip_data Dataframe
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286857 entries, 0 to 286856
Data columns (total 12 columns):
trip_id              286857 non-null int64
starttime            286857 non-null object
stoptime             286857 non-null object
bikeid               286857 non-null object
tripduration         286857 non-null float64
from_station_name    286857 non-null object
to_station_name      286857 non-null object
from_station_id      286857 non-null object
to_station_id        286857 non-null object
usertype             286857 non-null object
gender               181557 non-null object
birthyear            181553 non-null float64
dtypes: float64(2), int64(1), object(9)
memory usage: 26.3+ MB


**The trip dataset contains 181,557 trips where the user was a member and 105,300 trips where the user was a Short Term Pass Holder.**

In [162]:
#Counting the number of member and short_term pass holder trips in the dataset
trip_data['usertype'].value_counts()

Member                    181557
Short-Term Pass Holder    105300
Name: usertype, dtype: int64

**To create the member only dataset, I will:**

- **Extract all the rows from the original trip dataset where the column usertype contains the value 'Member'and store it in separate dataset.**


- **Drop any rows with null value in gender or birthyear column if any remain.**


- **Drop the usertype column entirely as it has become redundant for this dataset.**

In [181]:
#Extracting information about only the member trips and storing them in a separate Dataframe
member_only = trip_data['usertype'] == 'Member'
member_trip_data = trip_data.loc[member_only]

#Dropping the 4 rows with missing values in the birthyear column from the member trip data set
birthyear_present = member_trip_data['birthyear'].notnull()
member_trip_data = member_trip_data.loc[birthyear_present]

In [187]:
#Removing the usertype column from the member trip data set since it is redundant 
member_trip_data = member_trip_data.drop(['usertype'],axis = 1)

**The new dataset 'member_trip_data' does not contain any null values in any of it's columns.**

In [188]:
member_trip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181553 entries, 0 to 286848
Data columns (total 11 columns):
trip_id              181553 non-null int64
starttime            181553 non-null object
stoptime             181553 non-null object
bikeid               181553 non-null object
tripduration         181553 non-null float64
from_station_name    181553 non-null object
to_station_name      181553 non-null object
from_station_id      181553 non-null object
to_station_id        181553 non-null object
gender               181553 non-null object
birthyear            181553 non-null float64
dtypes: float64(2), int64(1), object(8)
memory usage: 16.6+ MB


In [189]:
member_trip_data.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,gender,birthyear
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Male,1960.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Male,1970.0
2,433,10/13/2014 10:33,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Female,1988.0
3,434,10/13/2014 10:34,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Female,1977.0
4,435,10/13/2014 10:34,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Male,1971.0


** Since the dataset does not contain gender and birthyear data for the trips taken by non members and so these columns only contain null values. To get a clean dataset for Temporary Pass Holders, I will:**

- **Extract all the rows from the original trip dataset where the usertype column contains the value 'Temporary Pass Holder'**


- **Drop birthname and gender column completely as they only contain null values for non members** 


- **Drop the usertype column as it has become redundant for this dataset**

In [197]:
#Extracting the information about the Short Term Pass Holder trips and storing them in a separate Dataframe 
pholder_only = trip_data['usertype'] == 'Short-Term Pass Holder'
pholder_trip_data = trip_data.loc[pholder_only]

In [194]:
#Dropping the colums containing all null values in the pass holder and the usertype column which has become redundant 
pholder_trip_data = pholder_trip_data.drop(['usertype','gender','birthyear'],axis = 1)

**The new datset 'pholder_trip_data' does not contain any null values in any of it's columns**

In [198]:
pholder_trip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105300 entries, 69 to 286856
Data columns (total 9 columns):
trip_id              105300 non-null int64
starttime            105300 non-null object
stoptime             105300 non-null object
bikeid               105300 non-null object
tripduration         105300 non-null float64
from_station_name    105300 non-null object
to_station_name      105300 non-null object
from_station_id      105300 non-null object
to_station_id        105300 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 8.0+ MB


In [196]:
pholder_trip_data.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id
69,507,10/13/2014 12:11,10/13/2014 12:16,SEA00321,332.457,City Hall / 4th Ave & James St,City Hall / 4th Ave & James St,CBD-07,CBD-07
78,518,10/13/2014 12:20,10/13/2014 12:31,SEA00321,690.793,City Hall / 4th Ave & James St,2nd Ave & Blanchard St,CBD-07,BT-05
89,530,10/13/2014 12:43,10/13/2014 12:48,SEA00311,278.849,King Street Station Plaza / 2nd Ave Extension ...,King Street Station Plaza / 2nd Ave Extension ...,PS-05,PS-05
91,533,10/13/2014 12:48,10/13/2014 12:51,SEA00269,179.939,NE 42nd St & University Way NE,12th Ave & NE Campus Pkwy,UD-02,UD-04
92,534,10/13/2014 12:48,10/13/2014 12:58,SEA00355,590.698,9th Ave N & Mercer St,Dexter Ave & Denny Way,DPD-01,SLU-18


## E. Exploratory Analysis & Story Telling